In [113]:
from io import StringIO
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from time import sleep
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException   
from selenium.common.exceptions import NoSuchWindowException   
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys

In [114]:
download_dir = "C:\\Users\\TFD\\PDF" 
options = webdriver.ChromeOptions()
profile = {"plugins.plugins_list": [{"enabled": False, "name": "Chrome PDF Viewer"}], # 關閉CHrome PDF Viewer
               "download.default_directory": download_dir , "download.extensions_to_open": "applications/pdf"}
options.add_experimental_option("prefs", profile)
driver = webdriver.Chrome("C:\\Users\\TFD\\Anaconda3\\chromedriver.exe", options=options) 
ActionChainsDriver = ActionChains(driver)
wait = WebDriverWait(driver,10)

In [115]:
driver.maximize_window()
driver.get("https://ssopxy.gov.taipei/SSOPXY/Default.aspx")
id =('j0470@tfd.gov.tw')
password = ('j180065#')

In [116]:
driver.find_element_by_name('TextBox_userID').send_keys(id)

In [117]:
driver.find_element_by_name('TextBox_userPwd').send_keys(password)

In [118]:
driver.find_element_by_name('Button_Login').click()

In [119]:
driver.find_element_by_class_name('category-button1').click()

In [120]:
driver.find_element_by_css_selector('input[value=新公文系統]').click()

In [121]:
child = driver.window_handles[1]      
driver.switch_to.window(child)
print(child)

CDwindow-E8F673D0B7DF3D88680B159010DC5C5E


In [122]:
driver.implicitly_wait(30)
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "toSaveCheck"))   
        
    )
finally:
    driver.find_element_by_id('toSaveCheck').click()

In [123]:
sleep(1)
driver.find_element_by_id('menu1487').click()

In [124]:
print (driver.current_url)

https://doc.gov.taipei/tcqb/home/default.jsp#


In [125]:
driver.switch_to.frame('dTreeContent')
driver.find_element_by_name('conditionQueryAll').click()

In [126]:
sleep(2)
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "q_IO_DEP1_NO"))   
    )
finally:
    driver.find_element_by_xpath("//select[@name='q_epaper2']/option[text()='線上']").click()

In [127]:
#"BDAW-第一救災救護大隊"
#"BDAX-第二救災救護大隊"
#"BDAY-第三救災救護大隊"
#"BDAZ-第四救災救護大隊"
driver.find_element_by_xpath("//select[@name='q_IO_DEP1_NO']/option[@value='BDAX']").click()
driver.find_element_by_xpath("//*[@id='qryModal']/div[2]/div/div[2]/div/div/div/table/tbody/tr[3]/td[4]/select/option[text()='電子收文']").click()

In [128]:
driver.find_element_by_name('q_ABSTRACT').clear()
driver.find_element_by_name('q_ABSTRACT').send_keys("貴公司")
driver.find_element_by_xpath('//*[@id="qryModal"]/div[2]/div/div[2]/div/div/div/table/tbody/tr[4]/td[2]/input[1]').send_keys('府')

In [129]:
driver.find_element_by_name('querySubmit').click()

In [130]:
def select_document():
    sleep(1)
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')
    driver.find_element_by_id('table_f7').click()
    driver.find_element_by_id('table_f7').click()
    driver.find_element_by_xpath('//*[@id="listTBODY"]/tr[1]/td[1]').click()
    driver.find_element_by_name('000091902').click()

In [131]:
def edit_case_number():
    def check_maximum_number():
        if  driver.find_element_by_xpath('//*[@id="listTBODY"]/tr/td[6]').get_attribute('innerHTML') == "999":
            m = driver.find_element_by_xpath('//*[@id="listTBODY"]/tr/td[5]').get_attribute('innerHTML')
            driver.find_element_by_xpath('//*[@id="listTBODY"]/tr/td[6]').click()
            driver.switch_to.window(driver.window_handles[1])
            driver.switch_to.frame('dTreeContent')
            driver.find_element_by_xpath('//*[@id="u_portfoliono"]').clear()
            driver.find_element_by_xpath('//*[@id="u_portfoliono"]').send_keys(int(m)+1)
            driver.find_element_by_xpath('//*[@id="updateBig"]').click()
            driver.switch_to.window(driver.window_handles[2])
            driver.find_element_by_xpath('//*[@id="queryTitle"]/table/tbody/tr[7]/td[2]/input').clear()
            driver.switch_to.alert.accept()
            driver.switch_to.window(driver.window_handles[2])
            driver.find_element_by_xpath('//*[@id="queryTitle"]/table/tbody/tr[8]/td[2]/input').clear()
            driver.find_element_by_xpath('//*[@id="queryTitle"]/table/tbody/tr[7]/td[2]/input').send_keys(int(m)+1)
            driver.find_element_by_xpath('//*[@id="form1"]/table/tbody/tr[2]/td/input[11]').click()
            sleep(1)
            driver.close()
            print("已修改卷殼號")
            driver.switch_to.window(driver.window_handles[1])
            driver.switch_to.frame('dTreeContent')
            edit_case_number()
        else:
            driver.find_element_by_xpath('//*[@id="listTBODY"]/tr/td[6]').click()     
    driver.find_element_by_css_selector('#u_caseno').clear()
    driver.find_element_by_css_selector('#u_portfoliono').clear()
    driver.find_element_by_xpath('//*[@id="queryTitle"]/table/tbody/tr[3]/td[4]/input[2]').click()
    driver.switch_to.window(driver.window_handles[2])
    #一大線上 //*[@id="listTBODY"]/tr[2]
    #二大線上 //*[@id="listTBODY"]/tr[4]
    #三大線上 //*[@id="listTBODY"]/tr[7]
    #四大線上 //*[@id="listTBODY"]/tr[10]
    driver.find_element_by_xpath('//*[@id="listTBODY"]/tr[4]/td[5]').click()
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')
    driver.find_element_by_xpath('//*[@id="query"]').click()
    sleep(1)
    driver.switch_to.window(driver.window_handles[2])
    check_maximum_number()

In [132]:
def get_PDF_string():
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')
    driver.find_element_by_xpath('//*[@id="queryTitle"]/table/tbody/tr[5]/td[2]/input[3]').click()
    child3 = driver.window_handles[2]
    driver.switch_to.window(child3)
    sleep(3)
    def get_PDF_url():
        source = driver.page_source
        startPDF_url=source.find('blob:https://doc.')
        namePDF_url =source.find('blob:https://doc.gov.taipei/')
        EndPDF_url = source.find('">',startPDF_url)
        PDF_url =source[startPDF_url:EndPDF_url]
        PDF_name = source[startPDF_url+28:EndPDF_url]
        driver.get(PDF_url)
        sleep(3)
        return PDF_name
        
    def convert_pdf_to_txt(path):
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
        fp = open(path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos=set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
            interpreter.process_page(page)
        text = retstr.getvalue()
        fp.close()
        device.close()
        retstr.close()
        return text
    try:
        sleep(2)
        p = get_PDF_url()
    except:
        sleep(2)
        p = get_PDF_url()
    PDF_path = "PDF\\"+ p + ".pdf"
    return convert_pdf_to_txt(PDF_path)

In [133]:
def edit_case_name_by_PDF():
    companyName = pdfText[pdfText.find('正本：')+3:pdfText.find('副本：')]
    y = companyName.find('[')
    if y == -1:
        pass
    else: 
        companyName = companyName.replace(companyName[y:companyName.find(']')+1],"")
    print(companyName)
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')
    # 獲取原始案由
    text = driver.find_element_by_id('u_abstractMain').get_attribute('value')
    print(text)
    #處理貴公司字串
    text = text.replace('貴公司',companyName)
    
    #處理查照字串
    x= text.find("，復請查照")
    if x == - 1:
        x=text.find("，請查照")
        if x == -1:
            pass
        else :
            text = text.replace(text[x:x+4],"")
    else:
        text = text.replace(text[x:x+5],"")
    #處理統一編號
    x=text.find('（統一編號')
    if x == - 1:
        pass
    else:
        text = text.replace(text[x:text.find("）")+1],"")
    #處理隨案附告
    x=text.find("請務必")
    if x == -1:
        pass 
    else:
        text = text.replace(text[x:len(text)],"")
    #處理最後句號
    if text[len(text)-1] == '，':
        text = text[0:len(text)-1] +"。"
        
    print(text)
    driver.find_element_by_id('u_abstractMain').clear()
    driver.find_element_by_id('u_abstractMain').send_keys(text)

In [134]:
def check_number_of_attachment():
    driver.switch_to.window(driver.window_handles[2])
    driver.switch_to.default_content()
    source = driver.page_source
    z = 1
    for x in range(4,43):
        h = driver.find_element_by_id("ext-simplelistitem-"+str(x)).get_attribute('style')
        xh=h.find('-10000px')
        if xh == -1:
            print(str(z), end=',')
            z +=1
        else:
            break
    if z !=1:
        driver.switch_to.window(driver.window_handles[1])
        driver.switch_to.frame('dTreeContent')
        driver.find_element_by_name('u_mediunQty').clear()
        driver.find_element_by_name('u_mediunQty').send_keys(z)
        driver.switch_to.window(driver.window_handles[2])
        print("已修改為"+str(z)+"份附件")
    else:
        print('沒有附件要修改')
    driver.close()
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')


In [135]:
def check_insert_attach():
    driver.find_element_by_id('insertAttach').click()
    driver.switch_to.window(driver.window_handles[2])
    s = driver.find_element_by_xpath('//*[@id="listTBODY"]/tr/td').get_attribute('innerHTML')
    if s != "查無資料，請您重新輸入查詢條件！":
        driver.find_element_by_xpath('//*[@id="listTBODY"]/tr/td[2]').click()
        driver.find_element_by_xpath('//*[@id="spanDelete"]/input').click()
        driver.switch_to.alert.accept()
        driver.switch_to.window(driver.window_handles[2])
        print("已將登陸附件刪除")
    else: 
        print("確認無登錄附件")
    driver.close()
    driver.switch_to.window(driver.window_handles[1])
    driver.switch_to.frame('dTreeContent')

In [ ]:
for x in range (1,300):
    select_document()
    edit_case_number()
    pdfText = get_PDF_string()
    edit_case_name_by_PDF()
    check_number_of_attachment()
    check_insert_attach()
    driver.find_element_by_id('doConfirm').click()
    driver.find_element_by_id('close').click()
    print('已編目'+str(x)+'筆公文')
    print("--------------------------------------------------------分隔線--------------------------------------------------------")
   

國亨餐飲股份有限公司
貴公司（統一編號：50885201）申請公司遷址變更登記，准予登記。並請詳閱說明欄相關事項，以保障公司權益，請查照。
國亨餐飲股份有限公司申請公司遷址變更登記，准予登記。並請詳閱說明欄相關事項，以保障公司權益。
沒有附件要修改
確認無登錄附件
已編目1筆公文
--------------------------------------------------------分隔線--------------------------------------------------------
皇鷲企業有限公司
貴公司申請設立、聘任經理人，准予登記（統一編號：82806617）。並請詳閱說明欄相關事項，以保障公司權益，請查照。
皇鷲企業有限公司申請設立、聘任經理人，准予登記。並請詳閱說明欄相關事項，以保障公司權益。
沒有附件要修改
確認無登錄附件
已編目2筆公文
--------------------------------------------------------分隔線--------------------------------------------------------
陞暘國際股份有限公司代理人：曾光輝會計師
貴公司（統一編號：28469602）申請公司遷址、改選董事、監察人、修正章程變更登記，准予登記。並請詳閱說明欄相關事項，以保障公司權益，請查照。
陞暘國際股份有限公司代理人：曾光輝會計師申請公司遷址、改選董事、監察人、修正章程變更登記，准予登記。並請詳閱說明欄相關事項，以保障公司權益。
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,已修改為18份附件
確認無登錄附件
已編目3筆公文
--------------------------------------------------------分隔線--------------------------------------------------------
恬健康有限公司代理人：成昀達會計師
貴公司（統一編號：54941726）申請所營事業變更、公司遷址、修正章程變更登記，准予登記。並請詳閱說明欄相關事項，以保障公司權益，請查照。
恬健康有限公司代理人：成昀達會計師申請所營事業變更、公司遷址、修正章程變更登記，准予登記。並請詳閱說明欄

In [ ]:
driver.close()
driver.switch_to.window(driver.window_handles[0])
driver.close()
os.execl(sys.executable, os.path.abspath('auto_edit_documents.py'), *sys.argv) 